# Training a semantic segmentation model using PyTorch

In this tutorial, we will learn how to train a semantic segmentation model using PyTorch.

Before you begin, ensure that you have *PyTorch* installed. To install a compatible version of PyTorch, use the requirement file:

```sh
pip install -r requirements-torch-cuda.txt
```

At a high level, we will:

- Read a dataset and create a *'training'* split. For this example, we will use the `SemanticKITTI` dataset.
- Train a model. We will train a `RandLANet` model on the *'training'* split.
- Run a test on a *'test'* split to evaluate the model.
- Run an inference on a custom point cloud.


## Reading a dataset

Downloading scripts are available in: `Open3D-ML/scripts/download_datasets`

You can use any dataset available in the `ml3d.datasets` dataset namespace. Here, we will use the `SemanticKITTI` dataset and visualize it. You can use any of the other datasets to load data. However, you must understand that the parameters may vary for each dataset.

We will read the dataset by specifying its path and then get all splits.

In [ ]:
# Training Semantic Segmentation Model using PyTorch

# import torch
import open3d.ml.torch as ml3d

# Read a dataset by specifying the path. We are also providing the cache directory and training split.
dataset = ml3d.datasets.SemanticKITTI(dataset_path='SemanticKITTI/',
                                      cache_dir='./logs/cache',
                                      training_split=['00'],
                                      validation_split=['01'],
                                      test_split=['01'])

# Split the dataset for 'training'. You can get the other splits by passing 'validation' or 'test'
train_split = dataset.get_split('training')

#support of Open3d-ML visualizer in Jupyter Notebooks is in progress
#view the frames using the visualizer
#vis = ml3d.vis.Visualizer()
#vis.visualize_dataset(dataset, 'training',indices=range(len(train_split)))

Now that you have visualized the dataset for training, let us train the model.

## Training a model

First, import the desired model from `open3d.ml.torch.models`.

After you load a dataset, you can initialize any model and then train the model. The following example shows how you can train RandLANet:

In [ ]:
# Training Semantic Segmentation Model using PyTorch

# Import torch and the model to use for training
import open3d.ml.torch as ml3d
from open3d.ml.torch.models import RandLANet
from open3d.ml.torch.pipelines import SemanticSegmentation

# Read a dataset by specifying the path. We are also providing the cache directory and training split.
# dataset = ml3d.datasets.SemanticKITTI(dataset_path='/Users/sanskara/data/SemanticKITTI/', cache_dir='./logs/cache',training_split=['00'], validation_split=['01'], test_split=['01'])
dataset = ml3d.datasets.SemanticKITTI(dataset_path='SemanticKITTI/',
                                      cache_dir='./logs/cache',
                                      training_split=['00'],
                                      validation_split=['01'],
                                      test_split=['01'])

# Initialize the RandLANet model.
model = RandLANet(in_channels=3)
pipeline = SemanticSegmentation(model=model,
                                dataset=dataset,
                                max_epoch=3,
                                optimizer={'lr': 0.001},
                                num_workers=0)

# Run the training
pipeline.run_train()

The training checkpoints are saved in: `pipeline.main_log_dir` (default path is: “./logs/Model_Dataset/“). You can use them for testing and inference.

## Running a test

Next, we will evaluate the trained model on the test split by calling the `run_test()` method:

In [ ]:
pipeline.run_test()

## Running an inference

An inference processes point cloud and displays the results based on the trained model. For this example, we will use a trained `RandLANet` model.

This example gets the pipeline, model, and dataset based on our previous training example. It runs the inference based the "train" split and prints the results.

In [ ]:
# Get data from the SemanticKITTI dataset using the "test" split
train_split = dataset.get_split("test")
data = train_split.get_data(0)

# Run the inference
results = pipeline.run_inference(data)

# Print the results
print(results)